# ![](CoronavirusAnimation2.gif)

In [3]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from datetime import date, timedelta
from sklearn.cluster import KMeans
# from fbprophet import Prophet
# from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime

In [74]:
import folium
import pandas as pd
import numpy as np
df = pd.read_csv('covid-19-DistrictWiseCaseMap.csv')

m = folium.Map(
    location=[24.25135,89.91671],
    zoom_start=7
)
for index,rows in df.iterrows():
    if rows['lat'] and rows['lon']:
        folium.Marker([rows['lat'], rows['lon']], tooltip=rows['District']+" - "+str(rows['Cases'])).add_to(m)
folium.LayerControl().add_to(m)
m

***Bangladesh Covid-19 Case Summary Representation***

In [4]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
totalCaseNumber = dataFromCsv['Case'].sum()
totalDeathNumber = dataFromCsv['Death'].sum()
totalRecoveredNumber = dataFromCsv['Recovered'].sum()
totalPatientNumber = totalCaseNumber - (totalDeathNumber+totalRecoveredNumber)

slices = [totalPatientNumber,totalDeathNumber,totalRecoveredNumber]
activities = ['Patient','Death','Recovered']

dataFrame = {
    "slices" : slices,
    "activities" : activities
}
dataset = pd.DataFrame(dataFrame)
fig = px.pie(dataset,
             values="slices",
             names="activities",
             template="presentation",
             labels = {'slices' : 'No Cases', 'activities' : 'Status'},
             color_discrete_sequence=['#4169E1', '#DC143C', '#006400'],
             width=800,
             height=450,
             hole=0.6,
             title="Covid-19 Case Summary BD")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
py.offline.iplot(fig)

In [5]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
totalCaseNumber = int(dataFromCsv['Case'].iloc[-1])
totalDeathNumber = int(dataFromCsv['Death'].iloc[-1])
totalRecoveredNumber = int(dataFromCsv['Recovered'].iloc[-1])
totalPatientNumber = totalCaseNumber - (totalDeathNumber+totalRecoveredNumber)

slices = [totalPatientNumber,totalDeathNumber,totalRecoveredNumber]
activities = ['Patient','Death','Recovered']

dataFrame = {
    "slices" : slices,
    "activities" : activities
}
dataset = pd.DataFrame(dataFrame)
fig = px.pie(dataset,
             values="slices",
             names="activities",
             template="presentation",
             labels = {'slices' : 'No Cases', 'activities' : 'Status'},
             color_discrete_sequence=['#4169E1', '#006400', '#DC143C'],
             width=800,
             height=450,
             hole=0.6,
             title="Covid-19 Case Summary 24 Hours")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

In [6]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
totalDeathNumber = dataFromCsv['Death'].sum()
totalRecoveredNumber = dataFromCsv['Recovered'].sum()

slices = [totalDeathNumber,totalRecoveredNumber]
activities = ['Death','Recovered']

dataFrame = {
    "slices" : slices,
    "activities" : activities
}
dataset = pd.DataFrame(dataFrame)
fig = px.pie(dataset,
             values="slices",
             names="activities",
             template="presentation",
             labels = {'slices' : 'No Cases', 'activities' : 'Status'},
             color_discrete_sequence=['#DC143C', '#006400'],
             width=800,
             height=450,
             hole=0,
             title="Death VS Recovered Ratio")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
py.offline.iplot(fig)

In [7]:
# dataFromCsv = pd.read_csv('../Analysis Data/covid-19-PerDayCaseSummaryBD.csv')
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv',parse_dates=["Date"], index_col='Date')
dataFromCsv = dataFromCsv.cumsum()
dataFromCsv = dataFromCsv.reset_index()
# print(dataset)
# print(dataFromCsv)
datetimeobject = datetime.datetime.strptime(str(dataFromCsv['Date'].iloc[1]),'%Y-%m-%d %H:%M:%S')
startDate = datetimeobject.strftime('%d-%b-%Y')
datetimeobject = datetime.datetime.strptime(str(dataFromCsv['Date'].iloc[-1]),'%Y-%m-%d %H:%M:%S')
endDate = datetimeobject.strftime('%d-%b-%Y')
xlabel = str(str(startDate) + ' UPTO ' + str(endDate))
# print(xlabel)
fig = px.area(dataFromCsv, x="Date", y="Case", 
              template="presentation", 
              labels = {'Date': xlabel, 'Case': 'Positive Cases'},
              title='Postive Case Trend',
              color_discrete_sequence=['#4169E1'])
fig2 = px.area(dataFromCsv, x="Date", y="Death", 
               template="presentation",
               labels = {'Date': xlabel, 'Death': 'Death Cases'},
               title='Death Case Trend',
               color_discrete_sequence=['#DC143C'])

fig3 = px.area(dataFromCsv, x="Date", y="Recovered", 
               template="presentation",
               labels = {'Date': xlabel, 'Recovered': 'Recovered Cases'},
               title='Recovered Case Trend',
               color_discrete_sequence=['#FF7F50'])
# for i in fig.patches:
#     print(i)
fig.update_traces(textposition='top left',mode='lines+markers')
# fig.update_layout(annotations = [dict(text = str(dataFromCsv['Case'].iloc[-1]),x=10000,y=800)])
fig2.update_traces(textposition='top left',mode='lines+markers')
fig3.update_traces(textposition='top left',mode='lines+markers')
fig.show()
fig2.show()
fig3.show()

In [8]:
import plotly.graph_objects as go
dataset = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv',parse_dates=["Date"])
dataset = dataset.reset_index()
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='Positove', x=dataset['Date'], y=dataset['Case'], marker_color = '#4169E1'),
    go.Bar(name='Death', x=dataset['Date'], y=dataset['Death'], marker_color = '#DC143C'),
    go.Bar(name='Recovered', x=dataset['Date'], y=dataset['Recovered'], marker_color = '#006400')
])
# Change the bar mode
fig.update_layout(barmode='stack',template="presentation",title_text='Per Day Case Summary')
fig.show()

In [9]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
dataFromCsv = dataFromCsv[1:]
positiveRatio = round((dataFromCsv['Case'] / dataFromCsv['Tested'])*100,2)

positiveRatioDF = pd.Series(positiveRatio,name="Ratio")
datavsratio = pd.concat([dataFromCsv['Date'],positiveRatioDF], axis=1)

fig = px.bar(datavsratio, x='Date', y='Ratio',
             hover_data=['Date', 'Ratio'],
             color='Ratio',
             color_continuous_scale= "Reds",
             labels={'Ratio' : 'Positive Case Ratio(%)'},
             template="ggplot2",
             height = 600)
fig.show()

In [10]:
CoronaTestCaseNo = pd.read_csv('covid-19-TestResultsBD.csv')
# print(CoronaTestCaseNo)

iedrcCases = int(CoronaTestCaseNo['Lab'].iloc[1])
otherCases = int(CoronaTestCaseNo['OLab'].iloc[1])
slices = [iedrcCases,otherCases]
activities = ['In IEDCR','In Other Labs']
cols = ['#FF6347','#000080']
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Tests Conducted Total", "Tests Conducted 24 Hour"),
    specs=[[{"type": "domain"}, {"type": "domain"}]]
)
fig.add_trace(
    go.Pie(labels = activities,
           values=slices,
           marker_colors=['#FF6347','#000080'],pull=0.05, textinfo="percent+label"),
           row=1, col=1)

iedrcCases = int(CoronaTestCaseNo['Lab'].iloc[0])
otherCases = int(CoronaTestCaseNo['OLab'].iloc[0])
slices = [iedrcCases,otherCases]
activities = ['In IEDCR','In Other Labs']
cols = ['#FF6347','#000080']

fig.add_trace(
    go.Pie(labels = activities,
           values=slices,
           marker_colors=['#FF6347','#000080'],pull=0.05, textinfo="percent+label"),
           row=1, col=2)
fig.update_traces(hole=.4, hoverinfo="label+percent+value",textfont_size=15)
fig.update_layout(height=500, showlegend=True, template="presentation")

fig.show()

In [11]:
CoronaTestCaseNo = pd.read_csv('covid-19-TestResultsBD.csv')
# print(CoronaTestCaseNo)

postitiveCases = int(CoronaTestCaseNo['Total'].iloc[3])
negativeCases = int(CoronaTestCaseNo['Total'].iloc[1]) - postitiveCases
otherCases = int(CoronaTestCaseNo['OLab'].iloc[1])
slices = [postitiveCases,negativeCases]
activities = ['Covid-19 Positive','Covid-19 Negative']
cols = ['#DC143C','#4169E1']
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Test Results", "Test Results 24 Hour"),
    specs=[[{"type": "domain"}, {"type": "domain"}]]
)
fig.add_trace(
    go.Pie(labels = activities,
           values=slices,
           marker_colors=cols,pull=0.05,textinfo="percent+label"),
           row=1, col=1)

postitiveCases = int(CoronaTestCaseNo['Total'].iloc[2])
negativeCases = int(CoronaTestCaseNo['Total'].iloc[0]) - postitiveCases
otherCases = int(CoronaTestCaseNo['OLab'].iloc[1])
slices = [postitiveCases,negativeCases]
activities = ['Covid-19 Positive','Covid-19 Negative']
cols = ['#DC143C','#4169E1']

fig.add_trace(
    go.Pie(labels = activities,
           values=slices,
           marker_colors=cols,pull=0.05,textinfo="percent+label"),
           row=1, col=2)
fig.update_traces(rotation = 180,hole=0, hoverinfo="label+percent+value",textfont_size=13)
fig.update_layout(height=500, showlegend=True, template="presentation")

fig.show()

In [12]:
caseBangladesh = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
result = caseBangladesh.sort_values(['Tested'], ascending= True)
fig = px.area(dataFromCsv, x="Tested", y="Case", 
              template="presentation", 
              labels = {'Tested': 'Total Test', 'Case': 'Positive Cases'},
              title='Tested Vs Positive Cases Trend',
              color_discrete_sequence=['#4169E1'])

fig.update_traces(textposition='top left',mode='lines+markers')
# fig.update_layout(annotations = [dict(text = str(dataFromCsv['Case'].iloc[-1]),x=10000,y=800)])
fig.show()

In [13]:
caseBangladesh = pd.read_csv('covid-19-DhakaCityArea.csv')
caseBangladesh
result = caseBangladesh.sort_values(['Number'], ascending= True)
# print(result)
numberOfCases = result[-20:]
fig = px.bar(numberOfCases, x='Area', y='Number',
             hover_data=['Area', 'Number'],
             labels={'Area' : 'Affected Area', 'Number' : 'Number of Cases' },
             template="xgridoff",
             text='Number',
             title="Dhaka City Area Wise Covid-19 Cases(Top 20)",
             height = 600)
fig.show()
# numberOfCases

In [14]:
caseBangladesh = pd.read_csv('covid-19-DhakaCityArea.csv')
caseBangladesh
result = caseBangladesh.sort_values(['Number'], ascending= True)
numberOfCases = result[0:20]
fig = px.bar(numberOfCases, x='Area', y='Number',
             hover_data=['Area', 'Number'],
             labels={'Area' : 'Affected Area', 'Number' : 'Number of Cases' },
             template="xgridoff",
             text='Number',
             title="Dhaka City Area Wise Covid-19 Cases(Bottom 20)",
             range_y=[0,10],
             height = 600)
fig.show()

In [90]:
caseBangladesh = pd.read_csv('covid-19-DivisionWiseCase.csv')
# print(caseBangladesh)
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "table"}, {"type": "domain"}]]
)
# values=[caseBangladesh[k].tolist() for k in caseBangladesh.columns[1:]],
fig.add_trace(
    go.Table(
        header=dict(
            values=[str(caseBangladesh.columns[0]),str(caseBangladesh.columns[1])],
            font=dict(size=15),
            align="center"
        ),
        cells=dict(
            values=[caseBangladesh[k].tolist() for k in caseBangladesh.columns[0:]],
            align = "center")
    ),
    row=1, col=1
)

slices=caseBangladesh['Cases'].tolist()
activities=caseBangladesh['Division'].tolist()
# print(slices)
# print(activities)
fig.add_trace(
    go.Pie(labels = activities,
           values=slices,pull=0.05,hole=.5,textinfo="percent", rotation=180),
           row=1, col=2)
# fig.update_traces(rotation = 180,hole=0.5, hoverinfo="label+percent+value",textfont_size=13)
fig.update_layout(height=500, 
                  showlegend=True, 
                  template="presentation",
                  title_text="Division Wise Covid-19 Cases")
fig.show()



In [49]:
caseBangladesh = pd.read_csv('covid-19-DistrictWiseCase.csv')
dhaka=caseBangladesh[caseBangladesh['Division']=='Dhaka']
fig = px.treemap(dhaka, path=['Division', 'District', 'Cases'], values='Cases',
                  color='Cases', hover_data=['District'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(dhaka['Cases'], weights=dhaka['Cases']))
fig.update_layout(height=500, 
                  showlegend=True, 
                  template="presentation",
                  title_text="Dhaka Division Covid-19 Cases")
fig.show()


In [52]:
caseBangladesh = pd.read_csv('covid-19-DistrictWiseCase.csv')
Chattogram=caseBangladesh[caseBangladesh['Division']=='Chattogram']
fig = px.treemap(Chattogram, path=['Division', 'District', 'Cases'], values='Cases',
                  color='Cases', hover_data=['District'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(Chattogram['Cases'], weights=Chattogram['Cases']))
fig.update_layout(height=500, 
                  showlegend=True, 
                  template="presentation",
                  title_text="Chattogram Division Covid-19 Cases")
fig.show()

In [86]:

gender = pd.read_csv('covid-19-GenderWiseCase.csv')

fig = px.pie(gender, values='percentage', names='gender',hole=.5,color_discrete_sequence=px.colors.sequential.RdBu,)
fig.update_layout(
    title={
        'text': "Gender Wise Covid-19 Cases",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()



In [89]:
gender = pd.read_csv('covid-19-AgeWiseCase.csv')

fig = px.pie(gender, values='percentage', names='age_range',hole=.5,color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(
    title={
        'text': "Age Wise Covid-19 Cases",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [33]:
symptoms={'symptom':['Fever',
        'Dry cough',
        'Fatigue',
        'Sputum production',
        'Shortness of breath',
        'Muscle pain',
        'Sore throat',
        'Headache',
        'Chills',
        'Nausea or vomiting',
        'Nasal congestion',
        'Diarrhoea',
        'Haemoptysis',
        'Conjunctival congestion'],'percentage':[87.9,67.7,38.1,33.4,18.6,14.8,13.9,13.6,11.4,5.0,4.8,3.7,0.9,0.8]}

symptoms=pd.DataFrame(data=symptoms,index=range(14))
# symptoms

In [34]:
fig = px.bar(symptoms[['symptom', 'percentage']].sort_values('percentage', ascending=False), 
             y="percentage", x="symptom", color='symptom', 
             log_y=True, template='ggplot2', title='Symptom of  Coronavirus')
fig.show()

In [35]:
fig = px.pie(symptoms,
             values="percentage",
             names="symptom",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()